In [1]:
import requests
import json
from string import Template
import ast

# LLM Setup

## Ollama Zephyr

In [ ]:
def ollama_generate(input: str, model = "zephyr-beta"):
    url = "http://localhost:11434/api/generate"
    
    payload = json.dumps({
      "model": model,
      "prompt": input,
      "stream": False
    })
    headers = {
      'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    return json.loads(response.text)['response'].strip()

In [ ]:
chat_template = Template("""<|system|>
You are a friendly chatbot who always helpful, harmless and honest.</s>
<|user|>
$input</s>
<|assistant|>""")

prompt = chat_template.substitute(input="Hello!")
print(ollama_generate(prompt))

## OpenAI

In [3]:
from openai import OpenAI
import os

In [28]:
client = OpenAI()

def openai_generate(input: str, system_message = "You are an assistant who is always helpful, harmless and honest.", model = "gpt-3.5-turbo"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": input}
        ]
    )
    
    return completion.choices[0].message.content

# Tools

In [15]:
def web_search(input: str):
    # Use Serper API here
    return "Age of Tom Cruise is 42"

In [16]:
def meaning_of_life():
    return "42 is the answer to life, the universe, and everything."

In [17]:
def calculator():
    return "5"

# System State

In [18]:
system_state = {
    "task": "Calculate the double of the age of Tom Cruise",
    "tools": {
        "web_search": {
            "func": web_search,
            "description": "Useful to get information on an unknown topic by searching it online. Input must be a string.",
            # "return_direct": True
        },
        "meaning_of_life": {
            "func": meaning_of_life,
            "description": "Useful to get the meaning of life. No inputs needed for this tool.",
        },
        "calculator": {
            "func": calculator,
            "description": "Useful to perform basic arithmetic operations. Input must be a string.",
        },
    },
    "current_plan": [],
    "short_term_memory": [],
}

# Utility Functions

In [19]:
def get_current_tools():
    tool_str = ""
    for idx, tool in system_state['tools'].items():
        # print(idx, tool['description'])
        tool_str += f"Name: {idx}\nDescription: {tool['description']}\n\n"
    return tool_str.strip()

def get_current_step():
    return system_state['current_plan'][0]

def set_current_plan(plan: str):
    system_state['current_plan'] = ast.literal_eval(plan)
    print(system_state['current_plan'])

# Re-planner Agent

**Basic idea**: Based on Plan-and-execute type agents, but capable of more complex tasks as it is able to think and update the plan as and when needed. This helps it to tackle tasks with longer steps.

Prepare a plan based on the task. Loop over the below step to proceed,
1. `Think`: Keeping the current step in mind, choose the right tool from the arsenel with right input.
2. `Act`: Execute the selected tool with the input from the previous step.
3. `Observe`: Based on the output of the action step, extract relevant information.
4. `Update`: Based on the available data till now, re-think and update the plan if needed.

## Plan

In [29]:
planner_template = Template("""Your task is to analyze complex tasks and break them down into smaller sub-tasks.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Do not perform the task just yet, analyze and break down the task and return the sub-tasks.
* If a tool is used, sub-task must say something like, 'Use tool_a with X to do Y'.
* Use only the tools mentioned below if and when needed.
* Return the output in the form of an array, follow the below output format strictly.

Tools:
```
$tools
```

Output format:
```
[
    'sub-task 1', 
    'sub-task 2', 
    ...
]
```

Task: $input

Return the output in the specified format, do not deviate. Do not add any text before or after the output.
""")

prompt = planner_template.substitute(input="Calculate the double of the age of Tom Cruise", tools=get_current_tools())
# print(prompt)
set_current_plan(openai_generate(prompt))

['Use web_search with "Tom Cruise age" to find the age of Tom Cruise', 'Use calculator with "age * 2" to calculate the double of the age']


## Think

In [32]:
thought_template = Template("""Your task is to extract the tool name and the input to the tool for the current step in the specified format.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Understand the `Current step` and extract the tool name and the complete input to the tool.
* Refer to the list of tools available to get the right tool name.
* Follow the below output format strictly.

Current step: `$current_step`

Tools:
```
$tools
```

Output format:
```
{
    'current_step': "current step that is being analyzed",
    'tool_name': "name of the tool",
    'tool_input': ["input 1", "input 2", ...]
}
```

Return the output in the specified format, do not deviate. Do not add any text before or after the output.
""")

prompt = thought_template.substitute(current_step=get_current_step(), tools=get_current_tools())
# print(prompt)
action_inputs = openai_generate(prompt)

In [43]:
print(action_inputs)

{
    'current_step': 'Use web_search with "Tom Cruise age" to find the age of Tom Cruise',
    'tool_name': 'web_search',
    'tool_input': ['Tom Cruise age']
}


In [38]:
current_action = ast.literal_eval(action_inputs)
current_action['tool_name'], current_action['tool_input']

('web_search', ['Tom Cruise age'])

## Act

In [41]:
def execute_action(tool_name: str, tool_input: str):
    return system_state['tools'][tool_name]['func'](tool_input)

execute_action(current_action['tool_name'], current_action['tool_input'])

'Age of Tom Cruise is 42'

# Archive

## Zephyr templates

In [ ]:
planner_template = Template("""<|system|>
You are an assistant who is always helpful, harmless and honest.</s>
<|user|>
Your task is to analyze complex tasks and break them down into smaller sub-tasks.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Do not perform the task just yet, analyze and break down the task and return the sub-tasks.
* If a tool is used, sub-task must say something like, 'Use tool_a with X to do Y'.
* Use only the tools mentioned below if and when needed.
* The sub-tasks must not say 'store the data'. All the data is by default stored in memory.
* Return the output in the form of an array, follow the below output format strictly.

Tools:
```
$tools
```

Output format:
```
[
    'sub-task 1', 
    'sub-task 2', 
    ...
]
```

Task: $input

Return the output in the specified format, do not deviate. Do not add any text before or after the output.</s>
<|assistant|>""")

prompt = planner_template.substitute(input="Calculate the double of the age of Tom Cruise", tools=get_current_tools())
# print(prompt)
set_current_plan(ollama_generate(prompt))

In [ ]:
thought_template = Template("""<|system|>
You are an assistant who is always helpful, harmless and honest.</s>
<|user|>
Your task is to extract the tool name and the input to the tool for the current step in the specified format.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Understand the `Current step` and extract the tool name and the complete input to the tool.
* Refer to the list of tools available to get the right tool name.
* Follow the below output format strictly.

Current step: `$current_step`

Tools:
```
$tools
```

Output format:
```
{
    'current_step': "current step that is being analyzed",
    'tool_name': "name of the tool",
    'tool_input': ["input 1", "input 2", ...]
}
```

Return the output in the specified format, do not deviate. Do not add any text before or after the output.</s>
<|assistant|>""")

prompt = action_template.substitute(current_step=get_current_step(), tools=get_current_tools())
# print(prompt)
print(ollama_generate(prompt))